In [2]:
import torch

In [286]:
stride = (1, 1)
kernel_shape = (3, 3)

x = torch.arange(0, 25).float().resize(1,1,5,5).requires_grad_()
# ########
inch,rows,cols = x.shape[-3:]

newr = rows +1 - kernel_shape[0]
newc = cols +1 - kernel_shape[1]
x0 = x.transpose(-1,-3)

In [287]:
x

tensor([[[[ 0.,  1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.,  9.],
          [10., 11., 12., 13., 14.],
          [15., 16., 17., 18., 19.],
          [20., 21., 22., 23., 24.]]]], requires_grad=True)

In [288]:
z1 = x0.unfold(len(x.shape)-2, kernel_shape[1], stride[1])
z2 = z1.unfold(len(x.shape)-3,kernel_shape[0],stride[0])
y = z2.contiguous().view(*x.shape[:-3],-1,kernel_shape[0]*kernel_shape[1]*inch)

In [289]:
y

tensor([[[ 0.,  1.,  2.,  5.,  6.,  7., 10., 11., 12.],
         [ 5.,  6.,  7., 10., 11., 12., 15., 16., 17.],
         [10., 11., 12., 15., 16., 17., 20., 21., 22.],
         [ 1.,  2.,  3.,  6.,  7.,  8., 11., 12., 13.],
         [ 6.,  7.,  8., 11., 12., 13., 16., 17., 18.],
         [11., 12., 13., 16., 17., 18., 21., 22., 23.],
         [ 2.,  3.,  4.,  7.,  8.,  9., 12., 13., 14.],
         [ 7.,  8.,  9., 12., 13., 14., 17., 18., 19.],
         [12., 13., 14., 17., 18., 19., 22., 23., 24.]]],
       grad_fn=<ViewBackward>)

In [291]:
y.transpose(-1,-2)

tensor([[[ 0.,  5., 10.,  1.,  6., 11.,  2.,  7., 12.],
         [ 1.,  6., 11.,  2.,  7., 12.,  3.,  8., 13.],
         [ 2.,  7., 12.,  3.,  8., 13.,  4.,  9., 14.],
         [ 5., 10., 15.,  6., 11., 16.,  7., 12., 17.],
         [ 6., 11., 16.,  7., 12., 17.,  8., 13., 18.],
         [ 7., 12., 17.,  8., 13., 18.,  9., 14., 19.],
         [10., 15., 20., 11., 16., 21., 12., 17., 22.],
         [11., 16., 21., 12., 17., 22., 13., 18., 23.],
         [12., 17., 22., 13., 18., 23., 14., 19., 24.]]],
       grad_fn=<TransposeBackward0>)

In [217]:
z = torch.sum(y)
z.backward()
x.grad

tensor([[[[1., 2., 3., 2., 1.],
          [2., 4., 6., 4., 2.],
          [3., 6., 9., 6., 3.],
          [2., 4., 6., 4., 2.],
          [1., 2., 3., 2., 1.]],

         [[1., 2., 3., 2., 1.],
          [2., 4., 6., 4., 2.],
          [3., 6., 9., 6., 3.],
          [2., 4., 6., 4., 2.],
          [1., 2., 3., 2., 1.]],

         [[1., 2., 3., 2., 1.],
          [2., 4., 6., 4., 2.],
          [3., 6., 9., 6., 3.],
          [2., 4., 6., 4., 2.],
          [1., 2., 3., 2., 1.]]]])

In [248]:
from torch import nn
import numpy as np
class FoldConv2d(nn.Module):
  def __init__(self,inch,outch,kshape,stride=1):
    nn.Module.__init__(self)
    k=1
    for j in kshape: k*=j
    self.kernel = nn.parameter.Parameter(torch.Tensor(inch*k,outch))
    self.kshape = kshape
    self.reset_parameters()
    
  def reset_parameters(self):
    torch.nn.init.kaiming_uniform_(self.kernel, a=np.sqrt(5))
    
  def forward(self,x):
    """
    Assumes x is shape (...,inch,row,col)
    and requires grad
    """
    inch,rows,cols = x.shape[-3:]

    newr = rows +1 - self.kshape[0]
    newc = cols +1 - self.kshape[1]
    xshape = x.shape
    x = x.transpose(-1,-3)
    x = x.unfold(len(xshape)-2, self.kshape[1], stride[1])
    x = x.unfold(len(xshape)-3,self.kshape[0],stride[0])
    x = x.contiguous().view(*xshape[:-3],newr*newc,-1)
    return x @ self.kernel

In [252]:
f = FoldConv2d(3,2,(3,3))
# print(f.kernel)
x = torch.arange(0, 75).float().resize(3,5,5).requires_grad_()
y=f.forward(x)
z = torch.sum(y)
# print(z,y)
z.backward()
x.grad

tensor([[[ 1.0129e+00,  9.4311e-01,  1.0389e+00,  2.6008e-02,  9.5802e-02],
         [ 2.3070e+00,  2.2896e+00,  2.2554e+00, -5.1535e-02, -3.4129e-02],
         [ 3.1114e+00,  3.3871e+00,  3.1383e+00,  2.6860e-02, -2.4876e-01],
         [ 2.0985e+00,  2.4440e+00,  2.0994e+00,  8.5139e-04, -3.4456e-01],
         [ 8.0447e-01,  1.0975e+00,  8.8286e-01,  7.8394e-02, -2.1463e-01]],

        [[ 3.3158e-01,  5.4768e-02, -7.7577e-01, -1.1073e+00, -8.3054e-01],
         [-5.9559e-01, -1.1398e+00, -2.9244e+00, -2.3288e+00, -1.7846e+00],
         [-4.6305e-01, -7.1147e-01, -3.1206e+00, -2.6575e+00, -2.4091e+00],
         [-7.9463e-01, -7.6624e-01, -2.3448e+00, -1.5502e+00, -1.5786e+00],
         [ 1.3255e-01,  4.2833e-01, -1.9618e-01, -3.2873e-01, -6.2450e-01]],

        [[-7.0888e-01, -3.3841e-01,  4.3040e-01,  1.1393e+00,  7.6881e-01],
         [-8.5042e-01, -1.6704e-01,  3.2368e-01,  1.1741e+00,  4.9072e-01],
         [ 1.5861e-01,  2.5292e-01,  5.8179e-01,  4.2318e-01,  3.2887e-01],
        